# Notebook 4
This notebook is to append to or format anything in the master table

In [1]:
%pylab inline
from astropy.table import Table, hstack,vstack, Column, QTable
from astropy.io import fits
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import FK5
from astropy.time import Time
from astropy.coordinates import Angle
from astropy.io import ascii
import numpy.ma as ma

Populating the interactive namespace from numpy and matplotlib


In [2]:
### use the master table here 
tsc = Table.read('/Users/JanviMadhani/thaw_star_catalog/subtables/master.csv') ### change file path to your computer location

In [3]:
#reset datatypes so that they are all strings/objects
tscols = tsc.colnames
for name in tscols:
    tsc[name] = tsc[name].astype(object)

# SECTION 1: Incorporate Data Entry 2 ( Delta Pi Values)
Assuming a 1-to-1 match between TSC number and Delta Pi Values

In [ ]:
delta_pi_vals = Table.read('filepath_here') # <--- INSERT file path to csv file with one column with TSC number 
                                            # and second column with delta pi value



In [4]:
def find_matching_tsc_no(number):
    """
    Given a file with Thaw Numbers in one column,
    look for the corresponding entry in the TSC table
    
    Input parameters: Thaw number you are searching for in the TSC
    Returns: the TSC INDEX for the corresponding entry in the TSC
    """
    number = str(number)
    for i in range(len(tsc)):
        if tsc['Thaw Number'][i] == number:
            return i

In [5]:
# Check the names of the columns for both tables here, 
# as they must be exactly the same in the code in the next cell
print(delta_pi_vals.colnames)
print(tsc.colnames)

['Thaw Number', 'YPC Number', 'YPC Component', 'Thaw Name', 'YPC Name', 'BD Number', 'Thaw RA1900', 'Thaw Dec1900', 'Thaw RA1950', 'Thaw Dec1950', 'Precessed RA1950', 'Precessed Dec1950', 'YPC RA1900', 'YPC Dec1900', 'RA2000', 'Dec2000', 'Thaw Mag', 'YPC (B-V)', 'YPC V', 'Thaw Spectrum', 'YPC Spectrum', 'Thaw Rel. Parallax', 'Thaw Rel. Parallax Error', 'Delta Pi', 'AO Publication Year', 'Thaw Abs. Parallax', 'Yale Abs. Parallax', 'Yale Abs. Parallax Error', 'Yale Tot. PM', 'Yale PM Position Angle', 'No. of Thaw Observations', 'Thaw Footnotes', 'Index Card Match Entry', 'Index Card Field Name', 'Index Card Mag.', 'Index Card RA', 'Index Card Dec', 'Index Card Equinox', 'Observations']


In [ ]:
###fill out master table with delta pi values matching based on tsc number between the two files

for i in range(len(tsc)):
    dp_thaw_no = delta_pi_vals['Thaw Number'][i] #Change this name to the exact name of the column that 
                                                 #holds the thaw number in the delta pi file
    
    tsc_i = find_matching_tsc_no(dp_thaw_no) #give this number to the function, which gives you back the index
                                             #of the matching entry in the tsc table 
    if tsc_i:
        tsc[tsc_i]['Delta Pi'] = delta_pi_vals['Delta Pi'][i] #Change the name of the delta pi column
                                                              #to the exact name in the table


In [ ]:
#CHECK to make sure values filled out as expected
tsc.show_in_notebook()

# SECTION 2: Create Specific Tables from the Master Table


## FIRST, pick which columns you want to include in your smaller table from these ones

In [6]:
print(tsc.colnames)

['Thaw Number', 'YPC Number', 'YPC Component', 'Thaw Name', 'YPC Name', 'BD Number', 'Thaw RA1900', 'Thaw Dec1900', 'Thaw RA1950', 'Thaw Dec1950', 'Precessed RA1950', 'Precessed Dec1950', 'YPC RA1900', 'YPC Dec1900', 'RA2000', 'Dec2000', 'Thaw Mag', 'YPC (B-V)', 'YPC V', 'Thaw Spectrum', 'YPC Spectrum', 'Thaw Rel. Parallax', 'Thaw Rel. Parallax Error', 'Delta Pi', 'AO Publication Year', 'Thaw Abs. Parallax', 'Yale Abs. Parallax', 'Yale Abs. Parallax Error', 'Yale Tot. PM', 'Yale PM Position Angle', 'No. of Thaw Observations', 'Thaw Footnotes', 'Index Card Match Entry', 'Index Card Field Name', 'Index Card Mag.', 'Index Card RA', 'Index Card Dec', 'Index Card Equinox', 'Observations']


## NEXT, copy and paste the names into the list below to replace what is shown in the example

In [7]:
#Replace names of columns in the order you want them in your table, HERE
mcolnames = ['Thaw Number','YPC Number', 'YPC Component', 'Thaw Name','YPC Name','RA2000', 'Thaw Rel. Parallax', 
             'Thaw Rel. Parallax Error','AO Publication Year','Yale Abs. Parallax','Yale Abs. Parallax Error']


m_table = Table(names = mcolnames,dtype = [object for i in range(len(mcolnames)) ]) #creates the table from given 
                                                                                    #column names

for i in range(len(tsc)):
    row = []  
    for name in mcolnames:
        row.append(tsc[i][name])
    m_table.add_row(row)

## NEXT, display the table to make sure it looks like what you were expecting

In [8]:
m_table.show_in_notebook()

idx,Thaw Number,YPC Number,YPC Component,Thaw Name,YPC Name,RA2000,Thaw Rel. Parallax,Thaw Rel. Parallax Error,AO Publication Year,Yale Abs. Parallax,Yale Abs. Parallax Error
0,1,51.00,--,Sigma Andromedae,SIG And,00 18 19.7,12,8,1917,19.0,12.5
1,2,158.00,--,Delta Piscium,DEL Psc,00 48 40.9,12,7,1917,17.0,9.4
2,3,209.00,--,Epsilon Piscium,EPS Psc,01 02 56.5,23,8,1917,26.0,6.5
3,4,219.00,A,Mu Cassiopeiae,MU Cas,01 08 16.2,143,9,1917,133.0,1.7
4,5,368.00,--,Omicron Piscium,OMI Psc,01 45 23.6,12,6,1917,21.0,7.7
5,6,394.00,--,Beta Arietis,BET Ari,01 54 38.4,72,8,1917,67.0,7.8
6,7,567.00,--,39 Arietis,39 Ari,02 47 54.5,22,6,1917,20.0,1.9
7,8,897.00,A,37 Tauri,37 Tau,04 04 41.7,-22,9,1917,12.0,5.7
8,--,897.00,B,--,--,04 04 39.4,--,--,--,--,--
9,9,911.00,--,50 Persei,50 Per,04 08 36.6,43,8,1917,47.0,12.5


### IF, you need to delete a column, or add a column, go back and replace the column names or re-order them  and rerun all cells in this section until you get to this point, then move on to the next part

## NEXT, write your table to a file
Here I will include multiple formats to write a file. Uncomment the one you want to use, or uncomment all and write multiple version of the table. Make sure you are careful with the file name. It WILL overwrite the last file with the same name and extension. 
All files will be written to the main directory from this notebook.
<br>
You can find all other ascii supported formats here if the format you want is not included below.
https://docs.astropy.org/en/stable/io/ascii/

In [ ]:
filename = 'minitable_example' #CHANGE this to the name you want to call your file


"""
OPTION ONE
Write to a CSV file (comma separated values), best for further processing with python.
"""
ascii.write(m_table, filename + '.csv',fast_writer = False,format='csv',overwrite=True)

"""
OPTION TWO
Write to a .tex file, a bare bones LaTex file, WILL require extensive further formatting
This file will be written as 'filename_latex.tex' as to not confuse it with the other latex
table option
"""
ascii.write(m_table, filename + '_latex.tex', format='latex' ,overwrite=True) 

"""
OPTION THREE
Write to a .tex file, a slightly fancier AASTEX template, WILL still require further formatting
This file will be written as 'filename_aastex.tex' as to not confuse it with the other latex
table option
There are two versions of writing with an aastex style available, uncomment the one you prefer
"""
ascii.write(m_table,filename + '_aastex.tex',Writer=ascii.Latex, overwrite = True, latexdict=ascii.latex.latexdicts['AA'])
#ascii.write(m_table, filename + '_aastex.tex', format='aastex' ,overwrite=True)  #another 


"""
OPTION FOUR
Write to a html file, to publish a table online
"""
ascii.write(m_table, filename + '.html', format='html' ,overwrite=True) 




# SECTION THREE: for proofreading tables purposes
Either create a mini table with just the relevant columns and write it as a printable file (all except HTML files can be easily printed for paper proofreading), OR, display the table of interest in a browser screen by running the next few cells. This option will display it as a nicely formatted HTML table that you can then right click on and print out. 

In [ ]:
#Proofreading the main table?
tsc.show_in_browser()
#tsc.show_in_notebook() #uncomment this to display the table inline with a search bar

In [ ]:
#Proofreading the current mini table?
m_table.show_in_browser()
#m_table.show_in_browser() #uncomment this to display the table inline with a search bar